In [48]:
import pandas as pd
import re
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
import requests
from math import ceil


In [49]:
# This is the list of almos 7 thousands decision about wrogdoing in public management from 

START_URL = 'http://www.lexml.gov.br/busca/search?keyword=improbidade;f1-tipoDocumento=Jurisprud%C3%AAncia;;year=2008;year-max=2018'
response = requests.get(START_URL)
data_raw = BeautifulSoup(response.text, 'html.parser')

In [50]:
# Inspecting data structure 

data = data_raw.find(class_='docHit')
len(data)

20

In [51]:
# Every item in "docHit" class is a piece of all information we want from a judicial decision
# We have to loop over all pages with 20 decisions each and get this pieces util the loop breaks 
# First we build de function that get the pages

decisions =  []

def get_decision (url):
    response = requests.get(url) 
    data_raw = BeautifulSoup(response.text, 'html.parser')
    data = data_raw.find(class_='docHit')
    decisions.append(data)
      


In [52]:
%%time
# Then we buil the list of links, and go over all pages.
# itemCount is the number of hits. That will limit the number of interations 
url =[]
text = 'http://www.lexml.gov.br/busca/search?keyword=improbidade;f1-tipoDocumento=Jurisprud%C3%AAncia;year=2008;year-max=2018;startDoc='

itens = int(data_raw.select_one('#itemCount').text)

data_raw.select(".resultsHeader tr")[2].select(".right a")[-1]["href"]

for n in range(0, ceil(itens/20)):
    x = str ((20*n)+1)
    url.append(text+(x))
url[240]

with ThreadPoolExecutor(50) as pool:
       results = pool.map(get_decision, url)
    


CPU times: user 29.2 s, sys: 1.67 s, total: 30.8 s
Wall time: 28.3 s


In [53]:
#Inspecting the correcteness of decisions variable
len(decisions)

354

In [54]:
#decisions[0]
len(all_decisions)

7080

In [55]:
#Now we have to flatten the list in order to get the 20 docHits in each page at the same layer. 
all_decisions = []
for sublist in decisions:
    for val in sublist:
        all_decisions.append(val)

#len(decisions)
#len(all_decisions)
type(all_decisions)

list

In [56]:
#Here we will get the pieces of information we want about every decision

Numero_list=[]
Localidade_list =[]
Autoridade_list= []
Título_list=[]
Data_list=[]
Ementa_list=[]
URN_list=[]
Assuntos_list=[]


def get_data_from_DocHits (item_all_decision):

    Numero = item_all_decision.select(".col1 b")[0].text
    Numero_list.append(Numero)
    
    Localidade = item_all_decision.select(".col3")[0].text
    Localidade_list.append(Localidade)
    
    Autoridade = item_all_decision.select(".col3")[1].text
    Autoridade_list.append(Autoridade)

    Título = item_all_decision.select(".col3")[2].text
    Título_list.append(Título)
    
    Data = item_all_decision.select(".col3")[3].text
    Data_list.append(Data)

    Ementa = item_all_decision.select(".col3")[4].text
    Ementa_list.append(Ementa)

    URN = item_all_decision.select(".col3")[5].text
    URN_list.append(URN)

    Assuntos = item_all_decision.select(".col3")[6].text
    Assuntos_list.append(Assuntos)


In [57]:
%%time
# Now we paralelize the work of building the data extraction over de decisions

with ThreadPoolExecutor(50) as pool:
       results_final = pool.map(get_data_from_DocHits,all_decisions)


CPU times: user 8.54 s, sys: 339 ms, total: 8.88 s
Wall time: 8.61 s


In [58]:
Numero_list[1]

'142'

In [59]:
Ementa_list[1]

'JUSTA CAUSA. CONFIGURADA. Há que ser mantida a justa causa aplicada pelo empregador quando verificada a prática de ato de improbidade, ensejando o rompimento do pacto empregatício por quebra da fidúcia entre as partes contratantes, elementos intrínsecos e fundamentais ao vínculo empregatício. A conduta do reclamante de utilizar-se do veículo da reclamada, sem autorização da empresa, para percorrer 106km fora da rota estipulada para aquele dia, sem avisar previamente sua empregadora ou obter autorização para usar o veículo para fins próprios é, pois, revestida de gravidade suficiente para autorizar a dispensa por justa causa, nos termos do artigo 482, "a" da CLT.'

In [63]:
#build pandas columms for the data frame

Numero_list=pd.DataFrame(Numero_list)
Localidade_list =pd.DataFrame(Localidade_list)
Autoridade_list= pd.DataFrame(Autoridade_list)
Título_list=pd.DataFrame(Título_list)
Data_list=pd.DataFrame(Data_list)
Ementa_list=pd.DataFrame(Ementa_list)
URN_list=pd.DataFrame(URN_list)
Assuntos_list=pd.DataFrame(Assuntos_list)




In [66]:
type(Numero_list)

pandas.core.frame.DataFrame

In [69]:
# Build and check the table
DF = pd.concat([Numero_list,
                    Localidade_list,
                    Autoridade_list,
                    Título_list,
                    Data_list,
                    Ementa_list, 
                    URN_list,
                    Assuntos_list],
                     axis = 1, ignore_index=True)
DF.columns = ["Numero",
                   "Localidade",
                   "Autoridade",
                   "Título",
                   "Data",
                   "Ementa", 
                   "URN",
                   "Assuntos"]
DF

,Numero,Localidade,Autoridade,Título,Data,Ementa,URN,Assuntos
0,141,Distrito Federal,Tribunal de Justiça do Distrito Federal e dos ...,Acórdão nº 432340 do Processo nº20100020055566...,01/07/2010,AGRAVO DE INSTRUMENTO. AÇÃO DE IMPROBIDADE ADM...,urn:lex:br;distrito.federal:tribunal.justica.d...,"DEFERIMENTO, RETORNO, AUTOS, VARA CÍVEL, AÇÃO ..."
1,142,3ª Região - Minas Gerais,Tribunal Regional do Trabalho. 2ª Turma,Acórdão PJe TRT 3ª / Segunda Turma / 2015-03-03,03/03/2015,JUSTA CAUSA. CONFIGURADA. Há que ser mantida a...,urn:lex:br;justica.trabalho;regiao.3:tribunal....,JUSTA CAUSA - IMPROBIDADE
2,143,Brasil,Superior Tribunal de Justiça. Corte Especial,AgRg na AIA 32 / AM,04/05/2016,AÇÃO DE IMPROBIDADE ADMINISTRATIVA. COMPET...,urn:lex:br:superior.tribunal.justica;corte.esp...,"COMPETÊNCIA JURISDICIONAL, JUIZ SINGULAR, PRIM..."
3,144,Brasil,Superior Tribunal de Justiça. 1ª Turma,AgRg no Ag 969454 / RJ,12/08/2008,PROCESSUAL CIVIL E ADMINISTRATIVO. AÇÃO CIVIL ...,urn:lex:br:superior.tribunal.justica;turma.1:a...,JUSTA CAUSA - IMPROBIDADE
4,145,3ª Região - Minas Gerais,Tribunal Regional do Trabalho. 9ª Turma,Acórdão TRT 3ª / Nona Turma / 2012-09-25,25/09/2012,EMENTA: JUSTA CAUSA. IMPROBIDADE. O ato de imp...,urn:lex:br;justica.trabalho;regiao.3:tribunal....,JUSTA CAUSA - IMPROBIDADE
5,146,3ª Região - Minas Gerais,Tribunal Regional do Trabalho. 8ª Turma,Acórdão TRT 3ª / Oitava Turma / 2012-10-03,03/10/2012,EMENTA: DISPENSA POR JUSTA CAUSA. ATO DE IMPRO...,urn:lex:br;justica.trabalho;regiao.3:tribunal....,JUSTA CAUSA - IMPROBIDADE
6,147,3ª Região - Minas Gerais,Tribunal Regional do Trabalho. Turma Recursal ...,Acórdão TRT 3ª / Decima Primeira Turma / 2014-...,18/03/2014,EMENTA: RECURSO ORDINÁRIO. ATO DE IMPROBIDADE ...,urn:lex:br;justica.trabalho;regiao.3:tribunal....,JUSTA CAUSA - IMPROBIDADE
7,148,3ª Região - Minas Gerais,Tribunal Regional do Trabalho. 4ª Turma,Acórdão TRT 3ª / Quarta Turma / 2014-10-01,01/10/2014,EMENTA: DISPENSA POR JUSTA CAUSA - ATO DE IMPR...,urn:lex:br;justica.trabalho;regiao.3:tribunal....,JUSTA CAUSA - IMPROBIDADE
8,149,3ª Região - Minas Gerais,Tribunal Regional do Trabalho. 3ª Turma,Acórdão PJe TRT 3ª / Terceira Turma / 2014-01-31,31/01/2014,DISPENSA POR JUSTA CAUSA. ATESTADO MÉDICO ADUL...,urn:lex:br;justica.trabalho;regiao.3:tribunal....,JUSTA CAUSA - IMPROBIDADE
9,150,3ª Região - Minas Gerais,Tribunal Regional do Trabalho. Turma Recursal ...,Acórdão PJe TRT 3ª / Decima Primeira Turma / 2...,07/07/2014,RECURSO ORDINÁRIO. ATO DE IMPROBIDADE DO EMPRE...,urn:lex:br;justica.trabalho;regiao.3:tribunal....,JUSTA CAUSA - IMPROBIDADE


In [70]:
#save DF as csv
DF.to_csv("Decisions_from_LexML")